In [2]:
from pycocotools.coco import COCO 
from pycocotools.cocoeval import COCOeval

def calculate_coco_map(gt_json_path, pred_json_path):
    """
    Calculate the mAP score using COCO API.
    Args:
        gt_json_path (str): Path to ground truth json file (COCO format).
        pred_json_path (str): Path to prediction json file (COCO format).
    Returns:
        float: mAP score.
    """
    
    # Load ground truth data
    coco_gt = COCO(gt_json_path)
    
    # Load prediction data
    coco_pred = coco_gt.loadRes(pred_json_path)

    # Set up evaluation (use 'iou' method for mAP)
    coco_eval = COCOeval(coco_gt, coco_pred, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()  # This prints the mAP and other stats
    
    return coco_eval  # Index 0 corresponds to mAP @ IoU=0.5

In [3]:
def analyze_detection_errors(coco_eval, iou_thresh_index=0):
    from collections import defaultdict

    print("\n🔍 Detection Error Analysis:\n")

    false_positive_imgs = defaultdict(list)
    misaligned_imgs = defaultdict(list)
    seen = set()  # Track (image_id, category_id) pairs already processed

    for eval_img in coco_eval.evalImgs:
        if eval_img is None:
            continue

        image_id = eval_img['image_id']
        category_id = eval_img['category_id']
        key = (image_id, category_id)

        # Prevent duplicate processing
        if key in seen:
            continue
        seen.add(key)

        file_name = image_id
        dt_matches = eval_img['dtMatches'][iou_thresh_index]
        dt_scores = eval_img['dtScores']
        gt_ids = eval_img['gtIds']
        dt_ids = eval_img['dtIds']

        # --- Case 1: No GT but detections (likely false positive) ---
        if len(gt_ids) == 0 and len(dt_ids) > 0:
            false_positive_imgs[file_name].append({
                "category_id": category_id,
                "scores": dt_scores
            })

        # --- Case 2: Has GT but predictions are not matched ---
        if len(gt_ids) > 0 and len(dt_ids) > 0:
            unmatched = [score for m, score in zip(dt_matches, dt_scores) if m == 0]
            if unmatched:
                misaligned_imgs[file_name].append({
                    "category_id": category_id,
                    "unmatched_scores": unmatched
                })

    # Print likely false positives
    print("❌ False Positives (No GT but predictions):")
    for img, preds in false_positive_imgs.items():
        for pred in preds:
            print(f"Image: {img}")
            print(f" - Category: {pred['category_id']}")
            print(f" - Confidence Scores: {pred['scores']}")
            print(f" - Note: No ground truth, but detection exists.")
            print("-" * 50)

    # Print misaligned predictions
    print("\n⚠️ Misaligned Predictions (With GT but unmatched detections):")
    for img, preds in misaligned_imgs.items():
        for pred in preds:
            print(f"Image: {img}")
            print(f" - Category: {pred['category_id']}")
            print(f" - Unmatched Scores: {pred['unmatched_scores']}")
            print(f" - Note: GT exists but detection did not match (low IoU or misclass).")
            print("-" * 50)


In [6]:
#pred_json_path = 'coco_ensemble_predictions.json'  # Replace with your actual ground truth JSON path
#pred_json_path = 'D:/BTXRD-Code/Object-Detection/runs/detect/val5/predictions.json'  # Replace with your actual prediction JSON path
#pred_json_path = 'fused_output.json'  
pred_json_path = 'D:/BTXRD-Dataset/BTXRD-Yolo/val/labels/detect.json'
gt_json_path = 'D:/BTXRD-Dataset/BTXRD-Yolo/val/labels/coco_annotations.json'  # Replace with your actual prediction JSON path

coco_eval = calculate_coco_map(gt_json_path, pred_json_path)
analyze_detection_errors(coco_eval)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.358
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.621
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.390
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.374
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.425
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

NameError: name 'analyze_detection_errors' is not defined

In [8]:
import json

# File paths
detect_path = 'D:/BTXRD-Dataset/BTXRD-Yolo/val/labels/detect.json'
coco_path = r"D:\BTXRD-Dataset\BTXRD-Yolo\val\labels\coco_annotations.json"

# Load detect.json
with open(detect_path, 'r') as f:
    detect_data = json.load(f)

# Load coco_annotations.json
with open(coco_path, 'r') as f:
    coco_data = json.load(f)

# Get image_ids from COCO annotations
coco_image_ids = set(ann["image_id"] for ann in coco_data["annotations"])

# Filter detections where image_id exists in COCO annotations
filtered_image_ids = [img_id for img_id in coco_image_ids if img_id not in {det["image_id"] for det in detect_data}]


# Print result
for img_id in sorted(filtered_image_ids):
    print(img_id)

IMG000024
IMG000044
IMG000049
IMG000077
IMG000108
IMG000114
IMG000129
IMG000199
IMG000221
IMG000244
IMG000260
IMG000278
IMG000317
IMG000340
IMG000359
IMG000366
IMG000411
IMG000436
IMG000439
IMG000452
IMG000495
IMG000498
IMG000527
IMG000590
IMG000627
IMG000629
IMG000650
IMG000666
IMG000667
IMG000680
IMG000702
IMG000708
IMG000712
IMG000723
IMG000740
IMG000786
IMG000819
IMG000827
IMG000848
IMG000865
IMG000870
IMG000891
IMG000922
IMG000990
IMG001024
IMG001027
IMG001125
IMG001249
IMG001256
IMG001304
IMG001379
IMG001414
IMG001439
IMG001483
IMG001513
IMG001580
IMG001671
IMG001717
IMG001741
IMG001758
IMG001769
IMG001782
IMG001858
IMG001866
